In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

### groupby 집계 

In [2]:
# 샘플 데이터프레임 생성
idx = ['A', 'A', 'B', 'B', 'B', 'C', 'C', 'C','D', 'D', 'D', 'D', 'E', 'E', 'E']
col = ['col1', 'col2', 'col3']
data = np.random.randint(0,9, (15,3))
df1 = pd.DataFrame(data=data, index=idx, columns = col).reset_index()
df1

,index,col1,col2,col3
0,A,8,2,3
1,A,0,4,6
2,B,3,7,0
3,B,0,2,4
4,B,4,7,5
5,C,6,8,3
6,C,6,1,0
7,C,7,1,4
8,D,0,7,4
9,D,2,7,8


In [3]:
# groupby() : 특정 컬럼별 통계치 구하기
df1.groupby('index').mean()  # sum() 합계 | mean() 평균 | count() 개수 | val() 분산 | std() 표준편차 | max() 최대값 | min() 최소값

,col1,col2,col3
index,,,
A,4.000000,3.000000,4.500000
B,2.333333,5.333333,3.000000
C,6.333333,3.333333,2.333333
D,1.500000,6.500000,3.750000
E,6.000000,5.000000,3.333333


In [4]:
# 둘 이상의 통계치 동시에 구하기
df1.groupby('index').agg(['sum', 'mean'])

col1           col2           col3          
       sum      mean  sum      mean  sum      mean
index                                             
A        8  4.000000    6  3.000000    9  4.500000
B        7  2.333333   16  5.333333    9  3.000000
C       19  6.333333   10  3.333333    7  2.333333
D        6  1.500000   26  6.500000   15  3.750000
E       18  6.000000   15  5.000000   10  3.333333

In [5]:
# 둘 이상의 통계치 동시에 구할때 함수 각각 적용하기
df1.groupby('index').agg({'col1' : 'mean', 'col2' : 'median', 'col3' : ['var', 'size']})

col1   col2       col3     
           mean median        var size
index                                 
A      4.000000    3.0   4.500000    2
B      2.333333    7.0   7.000000    3
C      6.333333    1.0   4.333333    3
D      1.500000    7.0   9.583333    4
E      6.000000    6.0  16.333333    3

In [6]:
# 둘 이상의 통계치 동시에 구할 때 컬럼의 멀티인덱스 정리, 소수점도 정리
def flatten_cols(df):
    df.columns = [' / '.join(x) for x in df.columns.to_flat_index()]
    return df

df1.groupby('index').agg(['sum', 'mean']).pipe(flatten_cols).round(2)

,col1 / sum,col1 / mean,col2 / sum,col2 / mean,col3 / sum,col3 / mean
index,,,,,,
A,8,4.00,6,3.00,9,4.50
B,7,2.33,16,5.33,9,3.00
C,19,6.33,10,3.33,7,2.33
D,6,1.50,26,6.50,15,3.75
E,18,6.00,15,5.00,10,3.33


In [7]:
# 하나의 대표값이 아닌 여러개의 값을 데이터프레임으로 호출
df1.groupby('index').describe().T

index              A         B         C         D         E
col1 count  2.000000  3.000000  3.000000  4.000000  3.000000
     mean   4.000000  2.333333  6.333333  1.500000  6.000000
     std    5.656854  2.081666  0.577350  1.000000  1.000000
     min    0.000000  0.000000  6.000000  0.000000  5.000000
     25%    2.000000  1.500000  6.000000  1.500000  5.500000
     50%    4.000000  3.000000  6.000000  2.000000  6.000000
     75%    6.000000  3.500000  6.500000  2.000000  6.500000
     max    8.000000  4.000000  7.000000  2.000000  7.000000
col2 count  2.000000  3.000000  3.000000  4.000000  3.000000
     mean   3.000000  5.333333  3.333333  6.500000  5.000000
     std    1.414214  2.886751  4.041452  1.000000  2.645751
     min    2.000000  2.000000  1.000000  5.000000  2.000000
     25%    2.500000  4.500000  1.000000  6.500000  4.000000
     50%    3.000000  7.000000  1.000000  7.000000  6.000000
     75%    3.500000  7.000000  4.500000  7.000000  6.500000
     max    4.000000  7.000000  8.000000  7.000000  7.000000
col3 count  2.000000  3.000000  3.000000  4.000000  3.000000
     mean   4.500000  3.000000  2.333333  3.750000  3.333333
     std    2.121320  2.645751  2.081666  3.095696  4.041452
     min    3.000000  0.000000  0.000000  1.000000  1.000000
     25%    3.750000  2.000000  1.500000  1.750000  1.000000
     50%    4.500000  4.000000  3.000000  3.000000  1.000000
     75%    5.250000  4.500000  3.500000  5.000000  4.500000
     max    6.000000  5.000000  4.000000  8.000000  8.000000

### 사용자정의 함수 적용하기

In [8]:
# apply 메서드 활용, 각 그룹별 col1 상위 2위만 추출
def top(df1, n=2, col='col1'):
    return df1.sort_values(by=col)[-n:]
df1.groupby('index', group_keys=False).apply(top)

,index,col1,col2,col3
1,A,0,4,6
0,A,8,2,3
2,B,3,7,0
4,B,4,7,5
6,C,6,1,0
7,C,7,1,4
10,D,2,7,2
11,D,2,5,1
13,E,6,2,8
12,E,7,7,1


In [9]:
# 조금더 쉽게
def get_top2(x):
    return x.sort_values('col1').head(2)
df1.groupby('index').apply(get_top2)

index  col1  col2  col3
index                           
A     1      A     0     4     6
      0      A     8     2     3
B     3      B     0     2     4
      2      B     3     7     0
C     5      C     6     8     3
      6      C     6     1     0
D     8      D     0     7     4
      9      D     2     7     8
E     14     E     5     6     1
      13     E     6     2     8

In [10]:
# lambda 로도 가능
df1.groupby('index').apply(lambda x:x.sort_values('col1').head(2))

index  col1  col2  col3
index                           
A     1      A     0     4     6
      0      A     8     2     3
B     3      B     0     2     4
      2      B     3     7     0
C     5      C     6     8     3
      6      C     6     1     0
D     8      D     0     7     4
      9      D     2     7     8
E     14     E     5     6     1
      13     E     6     2     8

In [11]:
# 기존 집계함수 대신 사용자 정의 함수 적용
def iqr_func(x):
    q3, q1 = np.percentile(x, [75, 25])
    return q3 - q1
df1.groupby('index').agg(iqr_func)

,col1,col2,col3
index,,,
A,4.0,1.0,1.50
B,2.0,2.5,2.50
C,0.5,3.5,2.00
D,0.5,0.5,3.25
E,1.0,2.5,3.50


### 필터 및 변환

In [12]:
# filter() : 그룹화한 데이터에서 원하는 데이터 걸러냄
df1.groupby('index').filter(lambda x : x['index'].count() > 2)

,index,col1,col2,col3
2,B,3,7,0
3,B,0,2,4
4,B,4,7,5
5,C,6,8,3
6,C,6,1,0
7,C,7,1,4
8,D,0,7,4
9,D,2,7,8
10,D,2,7,2
11,D,2,5,1


In [13]:
# get_group() : 그룹화한 데이터에서 해당 데이터만 추출, loc랑 뭐가 다른건지?
df1.groupby('index').get_group('B')

,index,col1,col2,col3
2,B,3,7,0
3,B,0,2,4
4,B,4,7,5


In [14]:
# transform() : 데이터를 표준화, 종전 기존 데이터프레임 크기 유지
df2 = df1.copy()
df2['col1_mean'] = df2.groupby('index').col1.transform('mean')
df2

,index,col1,col2,col3,col1_mean
0,A,8,2,3,4.000000
1,A,0,4,6,4.000000
2,B,3,7,0,2.333333
3,B,0,2,4,2.333333
4,B,4,7,5,2.333333
5,C,6,8,3,6.333333
6,C,6,1,0,6.333333
7,C,7,1,4,6.333333
8,D,0,7,4,1.500000
9,D,2,7,8,1.500000


### groupby 인덱스 처리

In [15]:
# 기존 인덱스 유지, 아래의 reset_index() 붙인 거와 동일 (그래서 별 의미 없을수도)
df1.groupby('index', as_index=False).sum()

,index,col1,col2,col3
0,A,8,6,9
1,B,7,16,9
2,C,19,10,7
3,D,6,26,15
4,E,18,15,10


In [16]:
df1.groupby('index').sum().reset_index()

,index,col1,col2,col3
0,A,8,6,9
1,B,7,16,9
2,C,19,10,7
3,D,6,26,15
4,E,18,15,10


### Na 포함된 데이터 그룹화

In [17]:
# null 포함 샘플 데이터프레임 생성
df3 = df1.copy()
df3.loc[6, 'index'] = np.NaN
df3.loc[9, 'col1'] = np.NaN
df3

,index,col1,col2,col3
0,A,8.0,2,3
1,A,0.0,4,6
2,B,3.0,7,0
3,B,0.0,2,4
4,B,4.0,7,5
5,C,6.0,8,3
6,NaN,6.0,1,0
7,C,7.0,1,4
8,D,0.0,7,4
9,D,NaN,7,8


In [18]:
# 기본적으로 Nan 값은 집계함수에서 제외
df3.groupby('index').mean()

,col1,col2,col3
index,,,
A,4.000000,3.000000,4.500000
B,2.333333,5.333333,3.000000
C,6.500000,4.500000,3.500000
D,1.333333,6.500000,3.750000
E,6.000000,5.000000,3.333333


In [19]:
# 하지만 dropna=False 인 경우 집계 컬럼의 NaN 포함되어 계산 (산출이 되는 컬럼에는 변동 없음)
df3.groupby('index', dropna=False).mean()

,col1,col2,col3
index,,,
A,4.000000,3.000000,4.500000
B,2.333333,5.333333,3.000000
C,6.500000,4.500000,3.500000
D,1.333333,6.500000,3.750000
E,6.000000,5.000000,3.333333
NaN,6.000000,1.000000,0.000000


### 수치형 데이터 범주화

In [20]:
# 샘플 데이터프레임 생성
df4 = sns.load_dataset('titanic')
df4

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [21]:
# cut() : 지정 구간으로 나누기
df4['age_group'] = pd.cut(df4.age,  # 범주화할 컬럼
                          bins = [0, 20, 40.123, np.inf],   # 경계값 지정, 리스트가 아닌 int로 입력시 동일 크기 구간으로 나눔
                          right = True,  # 각 구간의 오른쪽 포함 (~초과 ~이하), False는 반대, *True | False
                          labels=['child', 'young', 'old'],  # 레이블 이름, bins 경계값보다 1 적거나, int와 동일해야 함
                          include_lowest = False,  # 최소값 포함
                          precision = 2  # 경계값 소수점 제한
                         )
df4[['age', 'age_group']].sample(10)

,age,age_group
353,25.00,young
367,NaN,NaN
289,22.00,young
886,27.00,young
424,18.00,child
644,0.75,child
17,NaN,NaN
540,36.00,young
138,16.00,child
641,24.00,young


In [22]:
# 동일 구간으로 나누기
df4['age_group2'] = pd.cut(df4.age, 3, precision=1)
df4.groupby('age_group2').size()

age_group2
(0.3, 26.9]     319
(26.9, 53.5]    345
(53.5, 80.0]     50
dtype: int64

In [23]:
# 경계 구간 확인
pd.cut(df4.age, 3, retbins = True)[1]

array([ 0.34042   , 26.94666667, 53.47333333, 80.        ])

In [24]:
# 같은 갯수로 나누기 
df4['age_group3'] = pd.qcut(df4.age, 3)
df4.groupby('age_group3').size()

age_group3
(0.419, 23.0]    246
(23.0, 34.0]     232
(34.0, 80.0]     236
dtype: int64

In [25]:
# 분류 및 요약
def summary(x):
    result = {
        'sum' : x.sum(),
        'count' : x.count(),
        'mean' : x.mean(),
        'variance' : x.var()
    }
    return result

df4.groupby('age_group')['age'].apply(summary).unstack()

,sum,count,mean,variance
age_group,,,,
child,2264.67,179.0,12.651788,45.545508
young,11280.50,385.0,29.300000,30.791927
old,7660.00,150.0,51.066667,66.837808


### 그룹별 각각 조회

In [26]:
# 그룹별로 각각 순회하며 조회
for idx, group in df1.groupby('index'):
    display(group.head())

,index,col1,col2,col3
0,A,8,2,3
1,A,0,4,6


,index,col1,col2,col3
2,B,3,7,0
3,B,0,2,4
4,B,4,7,5


,index,col1,col2,col3
5,C,6,8,3
6,C,6,1,0
7,C,7,1,4


,index,col1,col2,col3
8,D,0,7,4
9,D,2,7,8
10,D,2,7,2
11,D,2,5,1


,index,col1,col2,col3
12,E,7,7,1
13,E,6,2,8
14,E,5,6,1
